## Assignment 6: Data Cleaning

In [165]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',\
'londON_StockhOlm',\
\
'Budapest_PaRis', 'Brussels_londOn'],\
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],\
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],\
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',\
'12. Air France', '"Swiss Air"']})

In [166]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


#### Question 1

Some values in the the FlightNumber column are missing.   
These numbers are  meant to increase by 10 with each row so 10055 and 10075 need to be put in place.    
Fill in these missing numbers and make the column an integer column (instead of a float column).    

In [167]:
df.dtypes

From_To          object
FlightNumber    float64
RecentDelays     object
Airline          object
dtype: object

In [168]:
df['FlightNumber']

0    10045.0
1        NaN
2    10065.0
3        NaN
4    10085.0
Name: FlightNumber, dtype: float64

In [169]:
df.index

RangeIndex(start=0, stop=5, step=1)

#### Here, we are first filling NaN in FlightNumber column with previous value + 10

In [170]:
for row in df.index:
    # comparision operation '==' doesn't work when compared to np.NaN, so use np.isnan() function
    if np.isnan(df['FlightNumber'][row]):
        #if we use df.iloc[row][column] we get a copy of the value and hence can't be modified.
        #To modify a cell value in pandas, first get the column series, then modify cell using row number
        df['FlightNumber'][row] = df['FlightNumber'][row-1] + 10
    

C:\Users\91984\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [171]:
df['FlightNumber']

0    10045.0
1    10055.0
2    10065.0
3    10075.0
4    10085.0
Name: FlightNumber, dtype: float64

In [172]:
df['FlightNumber'].dtype

dtype('float64')

#### Here, we are changing the data type of FlightNumber column from float to int

In [173]:
#df['FlightNumber'].astype(int) returns a copy, hence over writing the series.
df['FlightNumber'] = df['FlightNumber'].astype(int)

In [174]:
df.dtypes

From_To         object
FlightNumber     int32
RecentDelays    object
Airline         object
dtype: object

#### Question 2

The From_To column would be better as two separate columns!   
Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values.    
Assign the correct column names to this temporary   DataFrame.    

#### Created a temporary DF with two columns

In [175]:
temp_df = pd.DataFrame(df['From_To'].str.split('_', expand=True)) 

In [176]:
temp_df

,0,1
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


#### Renamed the columns

In [177]:
temp_df.columns = ['From', 'To']

In [178]:
temp_df

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


#### Question 3

Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame.   
Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)    

In [179]:
temp_df['From'].apply(lambda s: s.title())

0      London
1      Madrid
2      London
3    Budapest
4    Brussels
Name: From, dtype: object

In [180]:
temp_df['From'] = temp_df['From'].apply(lambda s: s.title())

In [181]:
temp_df['To'] = temp_df['To'].apply(lambda s: s.title())

In [182]:
temp_df

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


#### Question 4

Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [183]:
df.drop('From_To', axis=1)

,FlightNumber,RecentDelays,Airline
0,10045,"[23, 47]",KLM(!)
1,10055,[],<Air France> (12)
2,10065,"[24, 43, 87]",(British Airways. )
3,10075,[13],12. Air France
4,10085,"[67, 32]","""Swiss Air"""


#### Permanently deleting 'From_To' column from DataFrame

In [184]:
df.drop('From_To', axis=1, inplace=True)

#### Inserting new columns from temporary DataFrame

In [185]:

df.insert (0, 'From', temp_df['From'])

In [186]:
df.insert (1, 'To', temp_df['To'])

In [187]:
df

,From,To,FlightNumber,RecentDelays,Airline
0,London,Paris,10045,"[23, 47]",KLM(!)
1,Madrid,Milan,10055,[],<Air France> (12)
2,London,Stockholm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075,[13],12. Air France
4,Brussels,London,10085,"[67, 32]","""Swiss Air"""


#### Question 5

In the RecentDelays column, the values have been entered into the DataFrame as a list.    
We would like each first value in its own column, each second value in its own column, and so on.    
If there isn't an Nth value, the value should be NaN.
    
    
Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc.   
and replace the unwanted RecentDelays column in df with delays.    

In [188]:
delay_Sr = df['RecentDelays']

#### Convert list column to string format so that the columns can be expanded using str.strip() method

In [189]:
def convert_list_to_str(l):
    s = ' '
    for i in l:
        s += str(i) + ' '
        
    return s

In [190]:
delay_Sr = delay_Sr.apply(convert_list_to_str)
delay_Sr

0        23 47 
1              
2     24 43 87 
3           13 
4        67 32 
Name: RecentDelays, dtype: object

In [191]:
delay_Sr = delay_Sr.str.strip()


In [192]:
delays = delay_Sr.str.split(' ', expand=True)
delays

,0,1,2
0,23,47,None
1,,None,None
2,24,43,87
3,13,None,None
4,67,32,None


#### Rename the newly created columns, and convert to integer

In [193]:
delays.columns = ['delay_1', 'delay_2', 'delay_3']

In [195]:
delays

,delay_1,delay_2,delay_3
0,23,47,None
1,,None,None
2,24,43,87
3,13,None,None
4,67,32,None


In [196]:
delays['delay_1'] = pd.to_numeric(delays['delay_1'], errors='coerce', downcast='integer')

In [197]:
delays['delay_2'] = pd.to_numeric(delays['delay_2'], errors='coerce', downcast='integer')
delays['delay_3'] = pd.to_numeric(delays['delay_3'], errors='coerce', downcast='integer')

In [198]:
delays

,delay_1,delay_2,delay_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


#### Drop old column and insert new columns

In [199]:
df.drop('RecentDelays', axis=1, inplace=True)

In [200]:
df.insert (3, 'delay_1', delays['delay_1'])

In [201]:
df.insert (4, 'delay_2', delays['delay_2'])
df.insert (5, 'delay_3', delays['delay_3'])

In [202]:
df

,From,To,FlightNumber,delay_1,delay_2,delay_3,Airline
0,London,Paris,10045,23.0,47.0,NaN,KLM(!)
1,Madrid,Milan,10055,NaN,NaN,NaN,<Air France> (12)
2,London,Stockholm,10065,24.0,43.0,87.0,(British Airways. )
3,Budapest,Paris,10075,13.0,NaN,NaN,12. Air France
4,Brussels,London,10085,67.0,32.0,NaN,"""Swiss Air"""
